<a href="https://colab.research.google.com/github/hammaad2002/CenterFusionModel/blob/main/CenterFusion%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recursive https://github.com/brandesjj/centerfusionpp

Cloning into 'centerfusionpp'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (294/294), done.
remote: Total 332 (delta 31), reused 328 (delta 28), pack-reused 0
Receiving objects: 100% (332/332), 4.54 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
import os
CFPP_ROOT = '/content/centerfusionpp'
os.environ['CFPP_ROOT'] = CFPP_ROOT

In [3]:
!cd {CFPP_ROOT}/src/lib/model/networks/DCNv2 && ./make.sh

/bin/bash: line 1: cd: /content/centerfusionpp/src/lib/model/networks/DCNv2: No such file or directory


In [4]:
!pwd

/content


In [12]:
# import shutil
# shutil.rmtree('/content/centerfusionpp/src/lib/model/networks/DCNv2')

In [13]:
%cd {CFPP_ROOT}/src/lib/model/networks
!pwd
!git clone https://github.com/jinfagang/DCNv2_latest

/content/centerfusionpp/src/lib/model/networks
/content/centerfusionpp/src/lib/model/networks
Cloning into 'DCNv2_latest'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 315 (delta 40), reused 32 (delta 32), pack-reused 268
Receiving objects: 100% (315/315), 1.43 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [24]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5f5ldsv2
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5f5ldsv2
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=e10339c3474420f116beedd022a335a5b2e465ac6f03cb9c4e076dbc165d02c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-0mz5u030/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
directory /content/centerfusionpp/src/lib/model/networks/DCNv2/src already exists
Out bin /content/centerfusionpp/src/lib/model/networks/DCNv2/result.out


In [29]:
%%cu
#include <iostream>
using namespace std;

int main() {
  cout << "Hello World!";
  return 0;
}

Hello World!


In [7]:
!pip install torch torchvision torchaudio

In [15]:
%cd {CFPP_ROOT}/src/lib/model/networks/DCNv2_latest
!pwd
! ./make.sh

/content/centerfusionpp/src/lib/model/networks/DCNv2_latest
/content/centerfusionpp/src/lib/model/networks/DCNv2_latest
rm: cannot remove '*.so': No such file or directory
rm: cannot remove 'build/': No such file or directory
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running build
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
building '_ext' extension
creating build
creating build/temp.linux-x86_64-cpython-310
creating build/temp.linux-x86_64-cpython-310/content
creating build/temp.linux-x86_64-cpython-310/content/centerfusionpp
creating build/temp.linux-x86_64-cpython-310/content/centerfusionpp/src
creating build/temp.linux-x86_64-cpython-310/content/centerfusionpp/src/lib
creating build/temp.linux-x86_64-cpython-310/conten

In [16]:
!python -V

Python 3.10.12


In [18]:
!pip install progress

  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9612 sha256=33b3e499f3422589bf384af3ffe4d44e3afe5f6c30ee3bb7a3c368eb1294dcde
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [21]:
!pip show onnx

In [19]:
!python3 /content/centerfusionpp/src/convert_onnx.py

Traceback (most recent call last):
  File "/content/centerfusionpp/src/convert_onnx.py", line 19, in <module>
    from model.model import create_model, load_model
  File "/content/centerfusionpp/src/lib/model/model.py", line 11, in <module>
    from .networks.dla import DLASeg
  File "/content/centerfusionpp/src/lib/model/networks/dla.py", line 16, in <module>
    from .base_model import BaseModel
  File "/content/centerfusionpp/src/lib/model/networks/base_model.py", line 5, in <module>
    from utils.pointcloud import generate_pc_box_hm
  File "/content/centerfusionpp/src/lib/utils/pointcloud.py", line 7, in <module>
    from nuscenes.utils.data_classes import RadarPointCloud
ModuleNotFoundError: No module named 'nuscenes'
